In [2]:
import numpy as np
import json
import os
import argparse
import pandas as pd
import logging
import telethon

# Choosing the seed channels

You need paid account to access the data in table form, but I'm not spending even a cent on this russian service:)
I copypasted the tables from webpage directly to [google spreadsheet](https://docs.google.com/spreadsheets/d/1J6oCBaG2JE5ChDI5E018pRGJ9c9cnOZCOtW0iOeMwHw/edit#gid=0) and then extract links from cells using [custom App Script](https://docs.google.com/spreadsheets/d/1J6oCBaG2JE5ChDI5E018pRGJ9c9cnOZCOtW0iOeMwHw/edit#gid=0)

I've selected top100 (by citation index) channels from News and Media section, top100 from Blogs and top100 from Politics. I chose citation index (which is based on forwards and mentions) because it reflects potential information diffusion through the whole network (and we'll rely on the measure of influence further on).

In [3]:
df_links_raw = pd.read_csv("/Users/katerynaburovova/PycharmProjects/dehumanization/data/links_raw.csv")

In [9]:
df_links_raw = df_links_raw[df_links_raw['Unnamed: 2']!='drop'].reset_index(drop=True)

We have 10 values that were not linked through the share link - those are private and can be accessed after joining and parsing all joined chats. I expect that they will appear in our referenced chat - we drop it for now.

In [ ]:
df_links_raw = df_links_raw[df_links_raw['name_handle']!='#VALUE!']

In [20]:
df_links_raw = (df_links_raw.drop(columns=['Unnamed: 2', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 3', 'Unnamed: 5']))

We don't have duplicates:

In [23]:
df_links_raw['name_handle'].value_counts()[:5]

rian_ru           1
government_rus    1
RKadyrov_95       1
SolovievLive      1
russica2          1
Name: name_handle, dtype: int64

In [24]:
list_of_names = df_links_raw['name_handle'].to_list()

['rian_ru',
 'bbbreaking',
 'rt_russian',
 'breakingmash',
 'readovkanews',
 'tass_agency',
 'bazabazon',
 'sashakots',
 'shot_shot',
 'uranews',
 'rbc_news',
 'moscowach',
 'wargonzo',
 'warfakes',
 'ssigny',
 'pdmnews',
 'moscowmap',
 'tv360',
 'truekpru',
 'kommersant',
 'infantmilitario',
 'RtrDonetsk',
 'sputniklive',
 'meduzalive',
 'navideovidno',
 'milinfolive',
 'izvestia',
 'new_militarycolumnist',
 'zvezdanews',
 'voenacher',
 'ostorozhno_novosti',
 'karaulny',
 'SIL0VIKI',
 'sncmag',
 'tvrain',
 'bbcrussian',
 'stormdaily',
 'anna_news',
 'smotri_media',
 'glavmedia',
 'rtvimain',
 'mnews_ru',
 'chtddd',
 'ENews112',
 'grey_zone',
 'mozhemobyasnit',
 'Ateobreaking',
 'ru2ch_news',
 'surf_noise1',
 'radiogovoritmsk',
 'varlamov_news',
 'milchronicles',
 'daokedao',
 'infomoscow24',
 'donbassr',
 'vzglyad_ru',
 'swodki',
 'lentachold',
 'ChDambiev',
 'sotaproject',
 'readovkaru',
 'DmitriySteshin',
 'denazi_UA',
 'ntvnews',
 'svarschiki',
 'mosnow',
 'lentadnya',
 'kozakricha

## Now we create json which we'll feed to the parsing sript.

In [25]:
dict = {"names": list_of_names}


In [26]:
json_object = json.dumps(dict, indent=4)

In [27]:
with open("/Users/katerynaburovova/PycharmProjects/dehumanization/data/channels_list.json", "w") as outfile:
    outfile.write(json_object)

In [28]:
# bs check
with open('/Users/katerynaburovova/PycharmProjects/dehumanization/data/channels_list.json', 'r') as openfile:
     data = json.load(openfile)
data['names'][0]

'rian_ru'

In [24]:
# import os
# import sys
# os.chdir(sys.path[0])
#
# if f"{'tmp'}.session" in os.listdir():
#     os.remove(f"{'tmp'}.session")